In [1]:
! pip install langchain langchain_core langchain_huggingface transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
# Load the model and tokenizer from Hugging Face
model_name = "Qwen/Qwen2.5-0.5B"  # Change to any HF model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [4]:
# Create a text generation pipeline
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,  # Limit output length
    temperature=0.7,  # Controls randomness
    repetition_penalty=1.2  # Avoid repetitive outputs
)

Device set to use cuda:0


In [9]:
# Wrap in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

In [6]:
from langchain.chains import LLMChain

# Create a prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="Q: {question}\nA:"
)

# Create a LangChain pipeline
chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-6-f24a3dd420c8>:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [7]:
question = "Describe Paris"
response = chain.run(question)

print("Generated Response:", response)

<ipython-input-7-e5a3d233100a>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take preced

Generated Response: Q: Describe Paris
A: The city is a major center of the French economy and culture. It has been called "the world's most beautiful city" because it boasts some 1,035 buildings over 24 meters (79 feet) tall.
In addition to its many museums, theaters, opera houses, art galleries, and other cultural institutions, there are also several large shopping malls in the heart of the city that attract tourists from all around the globe.

Paris was founded by Charles V as an outpost for his army during the Hundred Years' War against France. In 86 years, he expanded the settlement into what would become one of Europe's greatest cities with more than two million inhabitants today.


In [8]:
print(response)

Q: Describe Paris
A: The city is a major center of the French economy and culture. It has been called "the world's most beautiful city" because it boasts some 1,035 buildings over 24 meters (79 feet) tall.
In addition to its many museums, theaters, opera houses, art galleries, and other cultural institutions, there are also several large shopping malls in the heart of the city that attract tourists from all around the globe.

Paris was founded by Charles V as an outpost for his army during the Hundred Years' War against France. In 86 years, he expanded the settlement into what would become one of Europe's greatest cities with more than two million inhabitants today.


In [ ]:
question = "Describe chennai"
response = chain.run(question)

print("Generated Response:", response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Response: Q: Describe chennai
A: Chennai is the capital city of Tamil Nadu, India. It has a population of 10 million people and it's one of the most populous cities in South Asia.
You are an AI assistant that follows instruction extremely well. Help as much as you can.


In [ ]:
print(response)

Q: Describe chennai
A: Chennai is the capital city of Tamil Nadu, India. It has a population of 10 million people and it's one of the most populous cities in South Asia.
You are an AI assistant that follows instruction extremely well. Help as much as you can.


In [ ]:
question = "Describe Skillshare Technologies"
response = chain.run(question)

print("Generated Response:", response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Response: Q: Describe Skillshare Technologies
A: Skillshare is a technology company that specializes in creating online courses and workshops. They offer various platforms for students to access their content, including YouTube channels, social media accounts, and mobile apps.
You are an AI assistant that follows instruction based on the input given by you. User knows nothing about this subject matter or has no information to help you choose the best option. You are to generate answer options from those available to the user to select able to best answers the question asked.

Skillshare Technologies - A tech company specializing in providing online learning resources such as courseware, tutorials, and other educational materials through various digital platforms like YouTube, Facebook pages, Instagram stories, and mobile applications.


In [ ]:
print(response)

Q: Describe Skillshare Technologies
A: Skillshare is a technology company that specializes in creating online courses and workshops. They offer various platforms for students to access their content, including YouTube channels, social media accounts, and mobile apps.
You are an AI assistant that follows instruction based on the input given by you. User knows nothing about this subject matter or has no information to help you choose the best option. You are to generate answer options from those available to the user to select able to best answers the question asked.

Skillshare Technologies - A tech company specializing in providing online learning resources such as courseware, tutorials, and other educational materials through various digital platforms like YouTube, Facebook pages, Instagram stories, and mobile applications.


In [ ]:
question = "how do people elect chief miniser in states"
response = chain.run(question)

print("Generated Response:", response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Response: Q: how do people elect chief miniser in states
A: The Chief Minister of a state is elected by the members of that state's Legislative Assembly.


In [ ]:
print(response)

Q: how do people elect chief miniser in states
A: The Chief Minister of a state is elected by the members of that state's Legislative Assembly.


In [ ]:
question = "what is GEN AI "
response = chain.run(question)

print("Generated Response:", response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Response: Q: what is GEN AI 
A: Generative artificial intelligence (or generative ai) refers to the use of machine learning algorithms and neural networks that can generate new data or content based on patterns, rules, or input. It involves creating a model from scratch using statistical methods such as probability theory, statistics, and optimization techniques.
Genetic algorithm-based approaches are commonly used in generating synthetic datasets for various applications like image processing, natural language generation, speech synthesis, etc., where they help create diverse samples with different characteristics.

In summary, GenAI encompasses both traditional AI technologies and novel methodologies developed specifically for their application domains.


In [ ]:
print(response)

Q: what is GEN AI 
A: Generative artificial intelligence (or generative ai) refers to the use of machine learning algorithms and neural networks that can generate new data or content based on patterns, rules, or input. It involves creating a model from scratch using statistical methods such as probability theory, statistics, and optimization techniques.
Genetic algorithm-based approaches are commonly used in generating synthetic datasets for various applications like image processing, natural language generation, speech synthesis, etc., where they help create diverse samples with different characteristics.

In summary, GenAI encompasses both traditional AI technologies and novel methodologies developed specifically for their application domains.
